In [ ]:
import os
import json
import numpy
import lmdb
from tqdm import tqdm
import pickle
from dataset import CVLGLocalizedNarrativesDataset
cvlg_lmdb = '/s2_md0/v-kunyan/CVLG_data/coco_train.lmdb',
config_dict = {"path":cvlg_lmdb,
    "max_readers":1,
    "readonly":False,
    "lock":False,
    "readahead":False,
    "meminit":False,
    "map_size":214748364800}
env2 = lmdb.open(**config_dict) 
txnout = env2.begin(write=True)

cnt = 0
keys = set()
from utils import load_yaml
config = load_yaml("./config/caption_coco2017.yaml")
# print(config.dataset_config.cvlg_coco2017)
dataset = CVLGLocalizedNarrativesDataset(config.dataset_config.cvlg_coco2017, dataset_type="train")
print(len(dataset))
from dalle_pytorch import OpenAIDiscreteVAE
from tqdm import tqdm
vae = OpenAIDiscreteVAE().cuda()
for idx, sample in enumerate(tqdm(dataset, total=len(dataset))):
    code = vae.get_codebook_indices(sample["image"].unsqueeze(0).cuda())

    anno = json.loads(line.strip())
    dataset_id = anno["dataset_id"]
    ln_image_id = anno["image_id"]
#             print(dataset_id)
    key = f"{dataset_id}_{str(cnt).zfill(8)}_{str(ln_image_id).zfill(12)}".encode()
#             print(key)
    assert key not in keys
    keys.add(key)
#             keys.append(key)
    txnout.put(key,pickle.dumps(anno))
#             print(anno)
    cnt += 1
    if cnt % 1000 == 0:
        txnout.commit()
        env2.close()
        env2 = lmdb.open(**config_dict) 

        txnout = env2.begin(write=True)
txnout.put(b"keys",pickle.dumps(keys))
txnout.commit()
env2.close()
            # env2 = lmdb.open(
            #     '/s1_md0/v-kunyan/kunyan/kvlb/mmf_cache/data/datasets/localized_narratives/defaults/annotations/coco_train_localized_narratives.lmdb',
            #     max_readers=1,
            #     readonly=False,
            #     lock=False,
            #     readahead=False,
            #     meminit=False,
            #     map_size=214748364800,
            #     # writemap=True,
            #     # map_async=True
            # ) 